In [ ]:
%pip install langgraph langchain openai

In [ ]:
# APPROACH 1

from langgraph.graph.state import StateGraph
from sqlalchemy.sql._elements_constructors import over
from dotenv import load_dotenv
from openai import OpenAI
import os
from typing import TypedDict, Optional
from langgraph.graph import StateGraph, END
from langgraph.types import interrupt

load_dotenv(override=True)
openai = OpenAI()
google_api_key = os.getenv('GOOGLE_API_KEY')


# ===============================
# STATE
# ===============================
class AgentState(TypedDict):
    userInput: str
    requiresApproval: bool
    approved: bool
    result: Optional[str]


# ===============================
# NODES
# ===============================

def detect_risk(state: AgentState) -> AgentState:
    text = state["userInput"].lower()
    risky_keywords = ["delete", "remove", "transfer", "pay", "withdraw"]

    state["requiresApproval"] = any(k in text for k in risky_keywords)
    return state


def approval_node(state: AgentState) -> AgentState:
    if state["requiresApproval"] and not state["approved"]:
        # ⛔ PAUSE EXECUTION HERE
        approve = interrupt({
            "type": "approval_required",
            "message": "This action requires human approval",
            "action": state["userInput"]
        })
        state['approved'] = approve
    return state


def execute_action(state: AgentState) -> AgentState:
    state["result"] = f"✅ Executed safely: {state['userInput']}"
    return state


# ===============================
# GRAPH
# ===============================

builder = StateGraph(AgentState)

builder.add_node("detect_risk", detect_risk)
builder.add_node("approval_node", approval_node)
builder.add_node("execute_action", execute_action)

builder.set_entry_point("detect_risk")
builder.add_edge("detect_risk", "approval_node")
builder.add_edge("approval_node", "execute_action")
builder.add_edge("execute_action", END)

graph = builder.compile()


# ===============================
# RUN (WILL PAUSE)
# ===============================
initial_state = {
    "userInput": "Delete all customer records",
    "requiresApproval": False,
    "approved": False,
    "result": None
}

paused_state = graph.invoke(initial_state)
print(paused_state)

# ===============================
# RESUME AFTER HUMAN APPROVAL
# ===============================
paused_state["approved"] = True

final_state = graph.invoke(paused_state)
print(final_state)

{'userInput': 'Delete all customer records', 'requiresApproval': True, 'approved': False, 'result': None, '__interrupt__': [Interrupt(value={'type': 'approval_required', 'message': 'This action requires human approval', 'action': 'Delete all customer records'}, id='cde5f6e7230d7e08011c82608168db5d')]}
{'userInput': 'Delete all customer records', 'requiresApproval': True, 'approved': True, 'result': '✅ Executed safely: Delete all customer records'}


In [16]:
# APPROACH 2
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.state import StateGraph
from sqlalchemy.sql._elements_constructors import over
from dotenv import load_dotenv
from openai import OpenAI
import os
from typing import TypedDict, Optional
from langgraph.graph import StateGraph, END
from langgraph.types import Command, interrupt

load_dotenv(override=True)
openai = OpenAI()
google_api_key = os.getenv('GOOGLE_API_KEY')


# ===============================
# STATE
# ===============================
class AgentState(TypedDict):
    userInput: str
    requiresApproval: bool
    approved: bool
    result: Optional[str]


# ===============================
# NODES
# ===============================

def detect_risk(state: AgentState) -> AgentState:
    text = state["userInput"].lower()
    risky_keywords = ["delete", "remove", "transfer", "pay", "withdraw"]

    state["requiresApproval"] = any(k in text for k in risky_keywords)
    print("Risk detected:", state["requiresApproval"])
    return state


def approval_node(state: AgentState) -> AgentState:
    print("Entering approval node")
    if state["requiresApproval"] and not state["approved"]:
        print("Requesting human approval")
        # ⛔ PAUSE EXECUTION HERE
        approve = interrupt({
            "type": "approval_required",
            "message": "This action requires human approval",
            "action": state["userInput"]
        })
        print("Approval received:", approve)
        state['approved'] = approve
    print("Exiting approval node")
    return state


def execute_action(state: AgentState) -> AgentState:
    print("Executing action")
    state["result"] = f"✅ Executed safely: {state['userInput']}"
    return state


# ===============================
# GRAPH
# ===============================

builder = StateGraph(AgentState)

builder.add_node("detect_risk", detect_risk)
builder.add_node("approval_node", approval_node)
builder.add_node("execute_action", execute_action)

builder.set_entry_point("detect_risk")
builder.add_edge("detect_risk", "approval_node")
builder.add_edge("approval_node", "execute_action")
builder.add_edge("execute_action", END)

checkpointer = MemorySaver()
graph = builder.compile(checkpointer=checkpointer)

# ===============================
# RUN (WILL PAUSE)
# ===============================
initial_state = {
    "userInput": "Delete all customer records",
    "requiresApproval": False,
    "approved": False,
    "result": None
}

config = {"configurable": {"thread_id": "approval-123"}}
paused_state = graph.invoke(initial_state, config=config)
print(paused_state)

# ===============================
# RESUME AFTER HUMAN APPROVAL
# ===============================
# paused_state["approved"] = True

final_state = graph.invoke(Command(resume=True), config=config)
print(final_state)

Risk detected: True
Entering approval node
Requesting human approval
{'userInput': 'Delete all customer records', 'requiresApproval': True, 'approved': False, 'result': None, '__interrupt__': [Interrupt(value={'type': 'approval_required', 'message': 'This action requires human approval', 'action': 'Delete all customer records'}, id='1317214c8dec26b80fece864011159b5')]}
Entering approval node
Requesting human approval
Approval received: True
Exiting approval node
Executing action
{'userInput': 'Delete all customer records', 'requiresApproval': True, 'approved': True, 'result': '✅ Executed safely: Delete all customer records'}
